In [28]:
import os
import sys
import pandas as pd
#sys.path.append('./')
#sys.path.append('/Users/yushiqiu/Documents/surv-rcts')
#dataset = os.system('python3 /Users/yushiqiu/Documents/surv-rcts/dev/dev_dataset.py')
import dev.dev_dataset as dev
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc
dataset=dev.dataset(sys.argv[1])



In [29]:
a = dataset._interventions
b = a['Main Study']
index_control = b.loc[b ==dataset.control_arm].index
index_treatment = b.loc[b==dataset.treatment_arm].index
control_covariate = dataset.covariates.loc[index_control]
treatment_covariate = dataset.covariates.loc[index_treatment]

In [30]:
from auton_survival.preprocessing import Preprocessor
features_control = Preprocessor().fit_transform(control_covariate, cat_feats=dataset.cat_features, num_feats=dataset.num_features)
features_treatment = Preprocessor().fit_transform(treatment_covariate, cat_feats=dataset.cat_features, num_feats=dataset.num_features)
outcomes_control = dataset.outcomes.loc[index_control]
outcomes_treatment = dataset.outcomes.loc[index_treatment]

In [31]:
import numpy as np
# horizons = [0.25, 0.5, 0.75]
# times = np.quantile(dataset.outcomes.time[dataset.outcomes.event==1], horizons).tolist()
#times_treatment = np.quantile(outcomes_treatment.time[outcomes_treatment.event==1], horizons).tolist()
times = [365.0,730.0,1825.0]


In [32]:
import numpy as np
horizons = [0.25, 0.5, 0.75]
# times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

In [33]:
x_control, t_control, e_control = features_control.values, outcomes_control.time.values, outcomes_control.event.values

n = len(x_control)

tr_size = int(n*0.50)
vl_size = int(n*0.20)
te_size = int(n*0.30)

x_train_control, x_test_control, x_val_control = x_control[:tr_size], x_control[-te_size:], x_control[tr_size:tr_size+vl_size]
t_train_control, t_test_control, t_val_control = t_control[:tr_size], t_control[-te_size:], t_control[tr_size:tr_size+vl_size]
e_train_control, e_test_control, e_val_control = e_control[:tr_size], e_control[-te_size:], e_control[tr_size:tr_size+vl_size]

In [34]:
x_treatment, t_treatment, e_treatment = features_treatment.values, outcomes_treatment.time.values, outcomes_treatment.event.values

n = len(x_treatment)

tr_size = int(n*0.50)
vl_size = int(n*0.20)
te_size = int(n*0.30)

x_train_treatment, x_test_treatment, x_val_treatment = x_treatment[:tr_size], x_treatment[-te_size:], x_treatment[tr_size:tr_size+vl_size]
t_train_treatment, t_test_treatment, t_val_treatment = t_treatment[:tr_size], t_treatment[-te_size:], t_treatment[tr_size:tr_size+vl_size]
e_train_treatment, e_test_treatment, e_val_treatment = e_treatment[:tr_size], e_treatment[-te_size:], e_treatment[tr_size:tr_size+vl_size]

In [35]:
x_train = np.concatenate((x_train_control, x_train_treatment), axis=0)
x_test = np.concatenate((x_test_control, x_test_treatment), axis=0)
x_val = np.concatenate((x_val_control, x_val_treatment), axis=0)

t_train = np.concatenate((t_train_control, t_train_treatment), axis=0)
t_test = np.concatenate((t_test_control, t_test_treatment), axis=0)
t_val = np.concatenate((t_val_control, t_val_treatment), axis=0)

e_train = np.concatenate((e_train_control, e_train_treatment), axis=0)
e_test = np.concatenate((e_test_control, e_test_treatment), axis=0)
e_val = np.concatenate((e_val_control, e_val_treatment), axis=0)



(2639, 41) (1583, 41) (1054, 41)


## Need to put back time and event for the sake of brier

In [36]:
# convert outcomes_test back to dataframe in order to fit in eval_brier
outcomes_test = np.concatenate((e_test.reshape(len(t_test), 1), t_test.reshape(len(e_test),1)), axis=1)

outcomes_test = pd.DataFrame(outcomes_test, columns=['event', 'time'])

y_train = np.concatenate((e_train.reshape(len(t_train), 1), t_train.reshape(len(e_train),1)), axis=1)

### Setting the parameter grid


In [37]:
from sklearn.model_selection import ParameterGrid

In [38]:
param_grid = {'k' : [3, 4, 6],
              'distribution' : ['LogNormal', 'Weibull'],
              'learning_rate' : [ 1e-4, 1e-3,1e-2],
              'layers' : [ [], [100], [100, 100], [100,100,100]]
             }
params = ParameterGrid(param_grid)

# DSM

In [39]:
from auton_survival.models.dsm import DeepSurvivalMachines
from auton_survival.metrics import survival_regression_metric

# Write a function to evaluate brier score

In [40]:
def eval_brier(predictions, outcomes_test,horizons):
    from auton_survival.metrics import survival_regression_metric
    from sklearn.metrics import roc_auc_score, brier_score_loss
    from sksurv import metrics
    from sksurv.util import Surv
    from tabulate import tabulate
    test_uncensored_brs = []
    for i, horizon in enumerate(horizons):

        y_pos = outcomes_test.time>=horizon
        y_neg = (outcomes_test.time<horizon)&(outcomes_test.event)
    
        outcomes_uncensored  = outcomes_test.loc[y_pos|y_neg]

        y = np.zeros(len(outcomes_uncensored))

        y[outcomes_uncensored.time>=horizon] = 1
        y[outcomes_uncensored.time<horizon] = 0
        test_uncensored_brs.append(brier_score_loss(y, predictions[:, i][y_pos|y_neg]))
    return test_uncensored_brs

In [41]:
models_DSM = []
for param in params:
    model = DeepSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 layers = param['layers'])
    # The fit method is called to train the model
    model.fit(x_train, t_train, e_train, iters = 100, learning_rate = param['learning_rate'])
    temp_pred_DSM = model.predict_survival(x_test, times)
    temp_score_DSM = eval_brier(temp_pred_DSM, outcomes_test, times)[1]
    models_DSM.append([temp_score_DSM, model])
best_model_DSM = min(models_DSM)
model_DSM = best_model_DSM[1]

 33%|███▎      | 33/100 [00:04<00:09,  7.16it/s]

[[0.03629979331565244, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261CD3EFC40>], [0.03633710085149897, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4D2F460>], [0.03599768577929197, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261CEBEFBB0>], [0.03565210099883334, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4D2F040>], [0.03598717387791633, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4E1CA60>], [0.036218163766734035, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4D4B1C0>], [0.03589392734643434, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4E4C700>], [0.03600074198659407, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261D4E471F0>], [0.03575522126433169, <auton_survival.models.dsm.DeepSurvivalMachines object at 0x00000261CEBE50F0>], [0.03583863702088738, <auton_survival.models.dsm.DeepSurvivalMachines object at 

IndexError: invalid index to scalar variable.

# Inference 

In [46]:
out_risk = model_DSM.predict_risk(x_test, times)
out_survival = model_DSM.predict_survival(x_test, times)


# Evaluation

In [48]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [49]:
def evaluate(predictions,out_risk, outcomes_train, outcomes_test,horizons):

    from auton_survival.metrics import survival_regression_metric
    from sklearn.metrics import roc_auc_score, brier_score_loss
    from sksurv import metrics
    from sksurv.util import Surv
    from tabulate import tabulate
  
    test_uncensored_aucs = []
    test_uncensored_brs = []
    test_uncensored_cis = []
    #test_censored_eces = []
    test_censored_brs = []
    test_censored_auc = []
    test_censored_cis = []

    from lifelines import KaplanMeierFitter

    censoring_outcomes = outcomes_train.copy()
   
    censoring_outcomes.event = 1-censoring_outcomes.event
  
    censoring_distribution = KaplanMeierFitter().fit(censoring_outcomes.time,
                                                     censoring_outcomes.event)
   

    for i, horizon in enumerate(horizons):

        y_pos = outcomes_test.time>=horizon
        y_neg = (outcomes_test.time<horizon)&(outcomes_test.event)
    
        outcomes_uncensored  = outcomes_test.loc[y_pos|y_neg]

        y = np.zeros(len(outcomes_uncensored))

        y[outcomes_uncensored.time>=horizon] = 1
        y[outcomes_uncensored.time<horizon] = 0

    
        test_uncensored_aucs.append(roc_auc_score(y, predictions[:, i][y_pos|y_neg]))
        test_uncensored_brs.append(brier_score_loss(y, predictions[:, i][y_pos|y_neg]))
        #test_censored_eces.append(expected_calibration_error(predictions[:, 1], outcomes_test, horizon))
    
        et_test_control_uncensored = np.array([(outcomes_uncensored.iloc[i, 1], outcomes_uncensored.iloc[i, 0]) for i in range(len(outcomes_uncensored))],
                 dtype = [('e', bool), ('t', float)])
    
        et_train_control = np.array([(outcomes_train.iloc[i,1], outcomes_train.iloc[i,0]) for i in range(len(outcomes_train))],
                 dtype = [('e', bool), ('t', float)])
        
        et_test_control_censored = np.array([(outcomes_test.iloc[i, 1], outcomes_test.iloc[i, 0]) for i in range(len(outcomes_test))],
                 dtype = [('e', bool), ('t', float)])


        outcomes_uncensored  = outcomes_test.loc[y_pos|y_neg]
        out_risk_uncensored = out_risk[y_pos|y_neg]

        test_uncensored_cis.append(concordance_index_ipcw(et_train_control, et_test_control_uncensored, out_risk_uncensored[:,i], horizons[i])[0])
        
        weights = np.ones(outcomes_uncensored.shape[0])
        weights = censoring_distribution.predict(outcomes_uncensored.time.values)
        weights[outcomes_uncensored.time.values>=horizon] = censoring_distribution.predict(horizon)

        weights = 1./np.clip(weights, 1e-3, 1-1e-3)

        test_censored_auc.append(roc_auc_score(y, predictions[:, i][y_pos|y_neg], sample_weight=weights))                
        test_censored_brs.append(brier_score_loss(y, predictions[:, i][y_pos|y_neg], sample_weight=weights))
        test_censored_cis.append(concordance_index_ipcw(et_train_control, et_test_control_censored,out_risk[:,i], horizons[i])[0])
    # from auton_survival.metrics import survival_regression_metric

    # test_censored_brs = survival_regression_metric('brs', outcomes['test'], predictions['TEST'], horizons, outcomes['train'])
    # test_censored_auc = survival_regression_metric('auc', outcomes['test'], predictions['TEST'], horizons, outcomes['train'])

    to_print = []
    to_print.append(['TEST FOLD PERFORMANCE'])
    to_print.append(["BR on Uncensored Data:"] + test_uncensored_brs)
    to_print.append(["AUC on Uncensored Data:"] + test_uncensored_aucs)
    to_print.append(["C-index on Uncensored Data:"] + test_uncensored_cis)

    to_print.append(["Censoring Adjusted Brier Score:"] + list(test_censored_brs))
    to_print.append(["Censoring Adjusted AUC:"] + list(test_censored_auc))
    #to_print.append(["Censoring Adjusted ECE:"] + list(test_censored_eces))
    to_print.append(["Censoring Adjusted C-index:"] + list(test_censored_cis))

    # to_print.append(['TRAIN FOLD PERFORMANCE'])
    # train_censored_brs = metrics.brier_score(survival_train, survival_train, logistic(predictions['TRAIN']), horizons)[-1]
    # train_censored_auc = metrics.cumulative_dynamic_auc(survival_train, survival_train, 1-logistic(predictions['TRAIN']), horizons)[0]
    # to_print.append(["Censoring Adjusted Brier Score:"] + list(train_censored_brs))
    # to_print.append(["Censoring Adjusted AUC:"] + list(train_censored_auc))

    print(tabulate(to_print, headers=["METRIC"]+horizons))

In [50]:
# n = len(train)

# tr_size = int(n*0.50)
# vl_size = int(n*0.20)
# te_size = int(n*0.30)

# train =  pd.concat([outcomes_control.iloc[:tr_size], outcomes_treatment.iloc[:tr_size]])
# test = pd.concat([outcomes_control.iloc[-te_size:], outcomes_treatment.iloc[-te_size:]])
train =  pd.concat([outcomes_control.iloc[:len(t_train_control)], outcomes_treatment.iloc[:len(t_train_treatment)]])
test = pd.concat([outcomes_control.iloc[-len(t_test_control):], outcomes_treatment.iloc[-len(t_test_treatment):]])


#when evaluating, pass the outcomes into evaluate()
#pass both out_survival and out_risk
print('DSM')
evaluate(out_survival,out_risk,train, test,times) 


METRIC                               365.0      730.0     1825.0
-------------------------------  ---------  ---------  ---------
TEST FOLD PERFORMANCE
BR on Uncensored Data:           0.0191821  0.035499   0.262331
AUC on Uncensored Data:          0.705225   0.682114   0.62905
C-index on Uncensored Data:      0.702797   0.678892   0.617007
Censoring Adjusted Brier Score:  0.0190674  0.0351439  0.0881924
Censoring Adjusted AUC:          0.70561    0.682163   0.627595
Censoring Adjusted C-index:      0.702618   0.67858    0.657847


# DCM

In [19]:
from auton_survival.models.dcm import DeepCoxMixtures
from auton_survival.models.dcm.dcm_utilities import *
model_DCM = DeepCoxMixtures()
model_DCM.fit(x_train, t_train, e_train, val_data=(x_val,t_val, e_val),\
         iters = 20, learning_rate = 1e-4, batch_size=120,optimizer = 'SGD')

100%|██████████| 20/20 [00:03<00:00,  6.42it/s]


In [20]:
out_survival = model_DCM.predict_survival(x_test,times)
out_risk = 1-out_survival

In [21]:
print('DCM')
evaluate(out_survival,out_risk,train, test,times) 


METRIC                               365.0      730.0     1825.0
-------------------------------  ---------  ---------  ---------
TEST FOLD PERFORMANCE
BR on Uncensored Data:           0.019323   0.0358222  0.266921
AUC on Uncensored Data:          0.664252   0.630887   0.560227
C-index on Uncensored Data:      0.662255   0.628319   0.54883
Censoring Adjusted Brier Score:  0.0192059  0.0354565  0.0902999
Censoring Adjusted AUC:          0.664613   0.63085    0.554984
Censoring Adjusted C-index:      0.662236   0.628031   0.610961


# CPH

In [51]:
x_control, y_control = features_control, outcomes_control

n = len(x_control)

tr_size = int(n*0.50)
vl_size = int(n*0.20)
te_size = int(n*0.30)

x_train_control, x_test_control, x_val_control = x_control.iloc[:tr_size], x_control.iloc[-te_size:], x_control.iloc[tr_size:tr_size+vl_size]
y_train_control, y_test_control, y_val_control = y_control.iloc[:tr_size], y_control.iloc[-te_size:], y_control.iloc[tr_size:tr_size+vl_size]


In [52]:
x_treatment, y_treatment = features_treatment, outcomes_treatment

n = len(x_treatment)

tr_size = int(n*0.50)
vl_size = int(n*0.20)
te_size = int(n*0.30)

x_train_treatment, x_test_treatment, x_val_treatment = x_treatment.iloc[:tr_size], x_treatment.iloc[-te_size:], x_treatment.iloc[tr_size:tr_size+vl_size]
y_train_treatment, y_test_treatment, y_val_treatment = y_treatment.iloc[:tr_size], y_treatment.iloc[-te_size:], y_treatment.iloc[tr_size:tr_size+vl_size]


In [53]:
x_train = pd.concat((x_train_control, x_train_treatment), axis=0)
x_test = pd.concat((x_test_control, x_test_treatment), axis=0)
x_val = pd.concat((x_val_control, x_val_treatment), axis=0)

y_train = pd.concat((y_train_control, y_train_treatment), axis=0)
y_test = pd.concat((y_test_control, y_test_treatment), axis=0)
y_val = pd.concat((y_val_control, y_val_treatment), axis=0)


print(x_train.shape, x_test.shape, x_val.shape)


(2639, 41) (1583, 41) (1054, 41)


In [54]:
param_grid_CPH = {'l2' : [1e-3, 1e-4]}
params = ParameterGrid(param_grid_CPH)

In [55]:
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
models_cph = []
for param in params:
    model = SurvivalModel('cph', random_seed=2, l2=param['l2'])

    # The fit method is called to train the model
    model.fit(x_train, y_train)

    # Obtain survival probabilities for validation set and compute the Integrated Brier Score
    predictions_val = model.predict_survival(x_val, times)
  
    metric_val = survival_regression_metric('ibs', y_val, predictions_val, times, y_train)
    models_cph.append([metric_val, model])

# Select the best model based on the mean metric value computed for the validation set
metric_vals = [i[0] for i in models_cph]
first_min_idx = metric_vals.index(min(metric_vals))
model_cph = models_cph[first_min_idx][1]

In [56]:
out_survival_cph = model_cph.predict_survival(x_test, times)
out_risk_cph = 1-out_survival_cph

In [57]:
#when evaluating, pass the outcomes into evaluate()
#pass both out_survival and out_risk
print('CPH')
evaluate(out_survival_cph,out_risk_cph,train, test,times) 


METRIC                               365.0      730.0     1825.0
-------------------------------  ---------  ---------  ---------
TEST FOLD PERFORMANCE
BR on Uncensored Data:           0.0193108  0.0357163  0.259202
AUC on Uncensored Data:          0.678427   0.673095   0.636334
C-index on Uncensored Data:      0.676591   0.669858   0.612999
Censoring Adjusted Brier Score:  0.0191958  0.0353604  0.0916823
Censoring Adjusted AUC:          0.678702   0.673208   0.632953
Censoring Adjusted C-index:      0.676882   0.669907   0.65391


# RSF

In [36]:
param_grid_RSF = {'n_estimators' : [100, 300],
              'max_depth' : [3, 5],
              'max_features' : ['sqrt', 'log2']}
params_RSF = ParameterGrid(param_grid_RSF)
models_RSF = []
for param in params_RSF:
    model = SurvivalModel('rsf', random_seed=2, n_estimators=param['n_estimators'], max_depth=param['max_depth'], max_features=param['max_features'])
    model.fit(x_train, y_train)
    predictions_val = model.predict_survival(x_val, times)
    # print(predictions_val,'predictmval')
    metric_val = survival_regression_metric('ibs', y_val, predictions_val, times, y_train)
    models_RSF.append([metric_val, model])
metric_vals = [i[0] for i in models_RSF]
first_min_idx = metric_vals.index(min(metric_vals))
model = models_RSF[first_min_idx][1]

[[0.97211853 0.94504253 0.86859415]
 [0.97992076 0.96532312 0.91773728]
 [0.97761007 0.96056173 0.90254057]
 ...
 [0.97586511 0.94665307 0.87451009]
 [0.97858015 0.95838922 0.90133151]
 [0.97542622 0.95206743 0.88710458]] predictmval
[[0.97254279 0.94519397 0.87102253]
 [0.98062445 0.96594313 0.92007644]
 [0.97834742 0.96111197 0.90456746]
 ...
 [0.97330406 0.94705956 0.87415674]
 [0.97944104 0.96042514 0.90227997]
 [0.97482008 0.95101464 0.88345504]] predictmval
[[0.97259721 0.94808132 0.87722816]
 [0.97934947 0.96393618 0.91608664]
 [0.97893029 0.9619882  0.90972807]
 ...
 [0.97334185 0.94789743 0.87224302]
 [0.97864213 0.96088357 0.90283617]
 [0.97507646 0.95302678 0.88686549]] predictmval
[[0.97338038 0.94706994 0.87675115]
 [0.98059144 0.96576998 0.91918691]
 [0.97837743 0.9613772  0.90633693]
 ...
 [0.97241958 0.94480666 0.864588  ]
 [0.97884899 0.96051772 0.90076857]
 [0.97477538 0.95143796 0.88354042]] predictmval
[[0.96970161 0.94272866 0.87165743]
 [0.98144713 0.96918951 0.92

In [37]:
out_survival_RSF = model.predict_survival(x_test, times)
out_risk_RSF = 1-out_survival_RSF
print('RSF')
evaluate(out_survival_cph,out_risk_cph,train, test,times) 

METRIC                               365.0      730.0     1825.0
-------------------------------  ---------  ---------  ---------
TEST FOLD PERFORMANCE
BR on Uncensored Data:           0.0193108  0.0357163  0.259202
AUC on Uncensored Data:          0.678427   0.673095   0.636334
C-index on Uncensored Data:      0.676591   0.669858   0.612999
Censoring Adjusted Brier Score:  0.0191958  0.0353604  0.0916823
Censoring Adjusted AUC:          0.678702   0.673208   0.632953
Censoring Adjusted C-index:      0.676882   0.669907   0.65391
